# Quantum Convolutional Neural Network

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# Quantum Convolution Layer

In [2]:
# Convolution Layer1
def U1(theta1, theta2, wires):
    qml.RY(theta1, wires = wires[0])
    qml.RY(theta2, wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

In [3]:
# Convoluyion Layer 2
def U2(theta, wires):
    qml.CRY(theta, wires = [wires[0], wires[1]])

In [4]:
#Pooling Layer
def V(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

In [5]:
# Fully connected layer
def F(wires):
    qml.CNOT(wires = [wires[0], wires[1]])

In [19]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding
dev = qml.device('default.qubit', wires = 8)
@qml.qnode(dev)
def QCNN1(X, params): # 6 parameters needed
    
    if len(X) == 8:
        AngleEmbedding(X, wires = range(8))
    elif len(X) == 256:
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    
    # 1st convolutional layer
    for i in range (0,8,2):
        U1(params[0], params[1], wires = [i, i+1])
    for i in range (1,7,2):
        U1(params[0], params[1], wires = [i, i+1])
    # 1st pooling layer
    for i in range(0,8,2):
        V(params[2], wires = [i + 1, i])
        
    # 2nd convolution layer
    U1(params[3], params[4], wires = [0,2])
    U1(params[3], params[4], wires = [4,6])
    U1(params[3], params[4], wires = [2,4])
    
    # 2nd pooling layer
    V(params[5], wires = [2, 0])
    V(params[5], wires = [6, 4])
    
    #Fully Connected Layer
    F(wires = [0,4])
    return qml.expval(qml.PauliZ(4))

In [18]:
dev1 = qml.device('default.qubit', wires = 8)
@qml.qnode(dev1)
def QCNN2(X, params): # 4 parameters needed
    if len(X) == 8:
        AngleEmbedding(X, wires = range(8))
    elif len(X) == 256:
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    
    # 1st convolutional layer
    for i in range (0,8,2):
        U2(params[0], wires = [i, i+1])
    for i in range (1,7,2):
        U2(params[0], wires = [i, i+1])
    # 1st pooling layer
    for i in range(0,8,2):
        V(params[1], wires = [i + 1, i])
        
    # 2nd convolution layer
    U2(params[2], wires = [0,2])
    U2(params[2], wires = [4,6])
    U2(params[2], wires = [2,4])
    
    # 2nd pooling layer
    V(params[3], wires = [2, 0])
    V(params[3], wires = [6, 4])
    
    #Fully Connected Layer
    F(wires = [0,4])
    return qml.expval(qml.PauliZ(4))

# Training Quantum Circuits

In [29]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, circuit = QCNN1):
    predictions = [circuit(x, params) for x in X]
    return square_loss(Y, predictions)

def accuracy(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

In [9]:
def optimization1(X_train, Y_train):
    params = np.random.randn(6)
    steps = 200
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, circuit = QCNN1), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

def optimization2(X_train, Y_train):
    params = np.random.randn(6)
    steps = 200
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, circuit = QCNN2), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### Results

In [12]:
import tensorflow as tf
def data_load_and_process(classes = [0,1]):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    
    x_train_01 = x_train[x_train_filter_01]
    y_train_01 = y_train[x_train_filter_01]
    x_test_01 = x_test[x_test_filter_01]
    y_test_01 = y_test[x_test_filter_01]
    
    
    x_train= tf.cast(x_train_01, tf.float32)
    x_test=tf.cast(x_test_01, tf.float32)
    
    x_train = tf.image.resize(x_train[:], (256, 1)).numpy()
    x_test = tf.image.resize(x_test[:], (256, 1)).numpy()
    
    x_train, x_test = tf.squeeze(x_train), tf.squeeze(x_test)
                                                                            
    return x_train, x_test, y_train_01, y_test_01

In [17]:
x_train, x_test, y_train, y_test = data_load_and_process()

In [14]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 8)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

Training QCNN with amplitdue encoding.    

In [20]:
trained_params_QCNN1_amp = optimization1(x_train, y_train)

iteration:  0  cost:  0.5799244070608663
iteration:  10  cost:  0.3480113041917416
iteration:  20  cost:  0.14773295692853852
iteration:  30  cost:  0.10292875139292906
iteration:  40  cost:  0.10239959187513324
iteration:  50  cost:  0.04623878517486909
iteration:  60  cost:  0.06792291117609364
iteration:  70  cost:  0.12334696779974891
iteration:  80  cost:  0.05849950083614769
iteration:  90  cost:  0.148977197101996
iteration:  100  cost:  0.11801545626952423
iteration:  110  cost:  0.08464095161573164
iteration:  120  cost:  0.10080178667497157
iteration:  130  cost:  0.056427967525085804
iteration:  140  cost:  0.2540777200842888
iteration:  150  cost:  0.13229944937447605
iteration:  160  cost:  0.09029562066455059
iteration:  170  cost:  0.10411996322968903
iteration:  180  cost:  0.11147801383143617
iteration:  190  cost:  0.11052790428925657


In [21]:
trained_params_QCNN1_pca = optimization1(x_train_pca, y_train)

iteration:  0  cost:  0.7143454388686875
iteration:  10  cost:  0.4815208529665137
iteration:  20  cost:  0.40163747292468416
iteration:  30  cost:  0.40681878647122505
iteration:  40  cost:  0.5861909408339573
iteration:  50  cost:  0.4484766498741056
iteration:  60  cost:  0.5117695149377027
iteration:  70  cost:  0.41619544017102383
iteration:  80  cost:  0.2703728814414295
iteration:  90  cost:  0.3308741102831155
iteration:  100  cost:  0.5888570803407841
iteration:  110  cost:  0.47195322429200354
iteration:  120  cost:  0.4767377037242238
iteration:  130  cost:  0.4694737382112432
iteration:  140  cost:  0.39568514285269935
iteration:  150  cost:  0.3580749461094057
iteration:  160  cost:  0.41718498943117166
iteration:  170  cost:  0.4886419647039543
iteration:  180  cost:  0.5123714716773169
iteration:  190  cost:  0.46004326904541004


In [22]:
trained_params_QCNN2_amp = optimization2(x_train, y_train)

iteration:  0  cost:  0.4314084304349942
iteration:  10  cost:  0.26724085421128835
iteration:  20  cost:  0.3918395363660904
iteration:  30  cost:  0.473754895503634
iteration:  40  cost:  0.2683007006110869
iteration:  50  cost:  0.3476448778459034
iteration:  60  cost:  0.2199913958217036
iteration:  70  cost:  0.3802733718172898
iteration:  80  cost:  0.33423299412546187
iteration:  90  cost:  0.23537480581823847
iteration:  100  cost:  0.2408518068993868
iteration:  110  cost:  0.3358024108373905
iteration:  120  cost:  0.33671514207525766
iteration:  130  cost:  0.2858186061824559
iteration:  140  cost:  0.3879426885379223
iteration:  150  cost:  0.36863962549274454
iteration:  160  cost:  0.4546251035679306
iteration:  170  cost:  0.3234578044508488
iteration:  180  cost:  0.24956473113249836
iteration:  190  cost:  0.3355540469882403


In [23]:
trained_params_QCNN2_pca = optimization2(x_train_pca, y_train)

iteration:  0  cost:  1.0366340293189955
iteration:  10  cost:  0.8781091881885733
iteration:  20  cost:  0.37842157954014866
iteration:  30  cost:  0.6715904763814892
iteration:  40  cost:  0.7122920313022096
iteration:  50  cost:  0.653351634307137
iteration:  60  cost:  0.8195811045840954
iteration:  70  cost:  0.3946135040616154
iteration:  80  cost:  0.4775672636587857
iteration:  90  cost:  0.4807877997979647
iteration:  100  cost:  0.35421834848948824
iteration:  110  cost:  0.38838073498147396
iteration:  120  cost:  0.491659616331758
iteration:  130  cost:  0.436727157682012
iteration:  140  cost:  0.4926413581774047
iteration:  150  cost:  0.42926023460189033
iteration:  160  cost:  0.3864186359177387
iteration:  170  cost:  0.38978598544287335
iteration:  180  cost:  0.5565480755760531
iteration:  190  cost:  0.2547157753997454


In [25]:
predictions1_amp = [QCNN1(x, trained_params_QCNN1_amp) for x in x_test]
predictions1_pca = [QCNN1(x, trained_params_QCNN1_pca) for x in x_test_pca]
predictions2_amp = [QCNN2(x, trained_params_QCNN2_amp) for x in x_test]
predictions2_pca = [QCNN2(x, trained_params_QCNN2_pca) for x in x_test_pca]

In [30]:
accuracy1_amp = accuracy(predictions1_amp, y_test)
accuracy1_pca = accuracy(predictions1_pca, y_test)
accuracy2_amp = accuracy(predictions2_amp, y_test)
accuracy2_pca = accuracy(predictions2_pca, y_test)

print("accuracy of QCNN1 with Amplitude Encoding: ", accuracy1_amp)
print("accuracy of QCNN1 with Angle Encoding: ", accuracy1_pca)
print("accuracy of QCNN2 with Amplitude Encoding: ", accuracy2_amp)
print("accuracy of QCNN2 with Angle Encoding: ", accuracy2_pca)

accuracy of QCNN1 with Amplitude Encoding:  0.9044917257683215
accuracy of QCNN1 with Angle Encoding:  0.46335697399527187
accuracy of QCNN2 with Amplitude Encoding:  0.46335697399527187
accuracy of QCNN2 with Angle Encoding:  0.5650118203309693
